## Preparation

In [8]:
import pandas as pd
import numpy as np
import pickle
import configparser
from sqlalchemy import create_engine
from sqlalchemy.types import String, Integer
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from matplotlib import style
from matplotlib import pyplot as plt
plt.style.use('ggplot')
import statsmodels.formula.api as smf
style.use("fivethirtyeight")

In [9]:
username= 'root'
password= 'Norika1124!'
database= 'Capstone_KAKEN'
appid = 'HhV6SqJ0yYlgkp0a887V'
url = 'mysql+pymysql://' + username + ':' + password + '@localhost:3306/' + database + '?charset=utf8'
engine = create_engine(url)

# Parsing 1: Product data (df_product)

### sql_product

In [ ]:
### sql_product = '''
### SELECT * 
### FROM grantaward_product
### '''
### grantaward_product2_full = pd.read_sql_query(sql_product, engine) #(9705651, 9)
### grantaward_product2_full.head()
### 
### #4 minutes

In [ ]:
### pathname = "/Users/norika_machome/GitHub/Capstone_KAKEN"
### 
### df_product = grantaward_product2_full
### 
### # Write to csv
### df_product.to_csv(pathname + "/Data/Parsed_csv/" + "df_product" + ".csv", index = False)
### print("Written to:", pathname + "/Data/Parsed_csv/" + "df_product" + ".csv")

### Descriptive Statistics

In [38]:
df_product = pd.read_csv("/Users/norika_machome/GitHub/Capstone_KAKEN/Data/Parsed_csv/df_product.csv")

In [11]:
#df_product.info()
# 666.4MB+
# shape: (9705651,9)
# columns: ['index', 'awardnumber', 'product_type', 'sequence', 'reviewed', 'doi','journaltitle_ja', 'journaltitle_en', 'year']
df_product.sample(10)

,index,awardnumber,product_type,sequence,reviewed,doi,journaltitle_ja,journaltitle_en,year
2185757,2185757,23653246,journal_article,10.0,NaN,NaN,季刊教育法(エイデル研究所),NaN,2012.0
4197180,4197180,21592780,presentation,75.0,NaN,NaN,NaN,NaN,2011.0
8755870,8755870,17K17538,jointInternational,5.0,NaN,NaN,NaN,NaN,NaN
6843326,6843326,21520434,book,7.0,NaN,NaN,NaN,NaN,2010.0
8918836,8918836,14340189,publication,19.0,NaN,NaN,NaN,NaN,NaN
5650646,5650646,21530399,journal_article,29.0,True,NaN,"広島国際研究 Vol.15, No.15",NaN,2009.0
1394625,1394625,16K06189,jointInternational,29.0,NaN,NaN,NaN,NaN,NaN
4877854,4877854,16390202,journal_article,31.0,NaN,NaN,J Pharmacol Exp Ther. 309,NaN,2004.0
4929528,4929528,16K18623,journal_article,5.0,True,10.1038/srep24549,Scientific Reports,NaN,2016.0
6054150,6054150,20046012,presentation,29.0,NaN,NaN,NaN,NaN,2009.0


In [33]:
print('Columns with null values:\n', df_product.isnull().sum())
print("-"*10)


print('ration of missing value:\n', df_product.isnull().sum()/len(df_product))

print("-"*10)

df_product = df_product.fillna(0) 
print('Columns with null values:\n', df_product.isnull().sum())




Columns with null values:
 index                    0
awardnumber              0
product_type         94436
sequence             94436
reviewed           7412053
doi                8786625
journaltitle_ja    6109267
journaltitle_en    9612374
year               1446330
dtype: int64
----------
ration of missing value:
 index              0.000000
awardnumber        0.000000
product_type       0.009730
sequence           0.009730
reviewed           0.763684
doi                0.905310
journaltitle_ja    0.629455
journaltitle_en    0.990389
year               0.149019
dtype: float64
----------
Columns with null values:
 index              0
awardnumber        0
product_type       0
sequence           0
reviewed           0
doi                0
journaltitle_ja    0
journaltitle_en    0
year               0
dtype: int64


In [13]:
missing_percentile = 1 - (94436*100/9705651)
print("Only ", round(missing_percentile,2), "% of product_type are missing , so I will simply drop them.")

Only  0.03 % of product_type are missing , so I will simply drop them.


In [14]:
df_product = df_product[df_product['product_type'].notna()]
df_product = df_product[df_product.product_type != 'note'] # Note was remarks, so drop all too
df_product.shape

(9459956, 9)

In [15]:
print(df_product['product_type'].value_counts())


presentation          4422113
journal_article       3642861
publication            709699
book                   409160
jointInternational     108016
0                       94436
patent                  60972
symposium               12698
press                       1
Name: product_type, dtype: int64


In [ ]:
df_overview = df_product

In [37]:
df_overview['awardnumber'].value_counts().describe()
print("skew;", df_overview['awardnumber'].value_counts().skew())


count    579685.000000
mean         16.319132
std          28.638882
min           1.000000
25%           2.000000
50%           8.000000
75%          20.000000
max        2362.000000
Name: awardnumber, dtype: float64

In [16]:


print("skew;", df_overview['awardnumber'].value_counts().skew())




16H06276    2362
15H05871    2051
15K21731    2048
16H06279    2047
12NP0201    2011
            ... 
19H01546       1
22590816       1
22590803       1
22590795       1
21K10193       1
Name: awardnumber, Length: 579685, dtype: int64
AVERAGE output ; 16.319131942348
max ; 2362
min ; 1
median; 8.0
skew; 11.261711374125644


 https://kaken.nii.ac.jp/en/grant/KAKENHI-PROJECT-16H06276/

In [17]:
df_16H06276 = df_product[df_product['awardnumber']=='16H06276']

### Creating df for each product_type (Function, ongoing)

product_journal_article,
product_jointinternational,
product_presentation,
product_book,
product_patent,
product_symposium,
product_publication


In [ ]:
#def product_type_count(df, product_type_lst):
#    for i in product_type_lst:
#        new_df_name = "product_" + i
#        new_df = df[df['product_type']== i]
#        new_df = new_df[['awardnumber','product_type']]
#        new_col_name = i + "_freq_count"
#        new_df[new_col_name] = new_df.groupby('awardnumber')['product_type'].transform('count')
#        new_df = new_df[['awardnumber',new_col_name]]
#        new_df = new_df.drop_duplicates().set_index('awardnumber')
#        
#        new_df_dataframe = pd.DataFrame(new_df)
#        df.join(new_df_dataframe)
#
#    display(df)
#        #product_df_all = product_df_all.merge(new_df_dataframe, right )
#        #display(product_df_all)
    

In [ ]:
##product_type_list = ["journal_article","jointInternational","presentation", "book","patent","symposium", "publication","note" ]
#product_type_list = ["journal_article","jointInternational","note"]
#product_list = ["product_journal_article","product_jointinternational","product_presentation", "product_book","product_patent","product_symposium", "product_publication" ]
#product_type_count(df_product, product_type_list)


### Creating Dataframe manually

In [19]:
product_journal_article = df_product[df_product["product_type"]=="journal_article"]
product_journal_article =product_journal_article[['awardnumber','product_type']]
product_journal_article['journal_freq_count'] = product_journal_article.groupby('awardnumber')['product_type'].transform('count')
product_journal_article =product_journal_article[['awardnumber','journal_freq_count']]
product_journal_article = product_journal_article.drop_duplicates()
product_journal_article = product_journal_article.set_index("awardnumber")
product_journal_article

,journal_freq_count
awardnumber,
17J01289,1
17J01281,7
17J01276,1
17J01269,9
17J01266,3
...,...
18H03012,11
18H03011,8
18H03010,9


In [20]:
product_jointinternational = df_product[df_product["product_type"]=="jointInternational"]
product_jointinternational =product_jointinternational[['awardnumber','product_type']]
product_jointinternational['jointInt_freq_count'] = product_jointinternational.groupby('awardnumber')['product_type'].transform('count')
product_jointinternational =product_jointinternational[['awardnumber','jointInt_freq_count']]
product_jointinternational = product_jointinternational.drop_duplicates()
product_jointinternational = product_jointinternational.set_index("awardnumber")
product_jointinternational

,jointInt_freq_count
awardnumber,
17J01289,1
17J01281,2
17J01269,2
17J01209,1
17J01169,1
...,...
18H03055,3
18H03052,1
18H03036,1


In [21]:
product_presentation = df_product[df_product["product_type"]=="presentation"]
product_presentation =product_presentation[['awardnumber','product_type']]
product_presentation['presentation_freq_count'] = product_presentation.groupby('awardnumber')['product_type'].transform('count')
product_presentation =product_presentation[['awardnumber','presentation_freq_count']]
product_presentation = product_presentation.drop_duplicates()
product_presentation = product_presentation.set_index("awardnumber")
product_presentation.head()

,presentation_freq_count
awardnumber,
17J01289,1
17J01285,2
17J01281,11
17J01276,8
17J01269,11


In [22]:
product_book = df_product[df_product["product_type"]=="book"]
product_book =product_book[['awardnumber','product_type']]
product_book['book_freq_count'] = product_book.groupby('awardnumber')['product_type'].transform('count')
product_book =product_book[['awardnumber','book_freq_count']]
product_book = product_book.drop_duplicates()
product_book = product_book.set_index("awardnumber")
product_book.head()

,book_freq_count
awardnumber,
17J01281,1
17J01103,2
17J01095,1
17J01093,1
17J00989,1


In [23]:
product_patent = df_product[df_product["product_type"]=="patent"]
product_patent =product_patent[['awardnumber','product_type']]
product_patent['patent_freq_count'] = product_patent.groupby('awardnumber')['product_type'].transform('count')
product_patent =product_patent[['awardnumber','patent_freq_count']]
product_patent = product_patent.drop_duplicates()
product_patent = product_patent.set_index("awardnumber")
product_patent.head()

,patent_freq_count
awardnumber,
17J01281,1
17J01243,1
17J01119,2
17J01091,1
17J00931,3


In [24]:
product_symposium = df_product[df_product["product_type"]=="symposium"]
product_symposium =product_symposium[['awardnumber','product_type']]
product_symposium['symposium_freq_count'] = product_symposium.groupby('awardnumber')['product_type'].transform('count')
product_symposium =product_symposium[['awardnumber','symposium_freq_count']]
product_symposium = product_symposium.drop_duplicates()
product_symposium = product_symposium.set_index("awardnumber")
product_symposium.head()

,symposium_freq_count
awardnumber,
17J01122,1
17J01046,1
17J00899,1
17J00447,2
17J00096,2


In [25]:
product_publication = df_product[df_product["product_type"]=="publication"]
product_publication =product_publication[['awardnumber','product_type']]
product_publication['publication_freq_count'] = product_publication.groupby('awardnumber')['product_type'].transform('count')
product_publication =product_publication[['awardnumber','publication_freq_count']]
product_publication = product_publication.drop_duplicates()
product_publication = product_publication.set_index("awardnumber")
product_publication.head()

,publication_freq_count
awardnumber,
15540249,1
15540248,2
15540247,2
15540245,1
15540243,5


# Parsnig 2: Basic Information

### sql_main df_main, df_categoryc, df_categoryb

In [26]:
sql_main = '''
SELECT 
grantaward.awardnumber,
grantaward.startfiscalyear,
grantaward.endfiscalyear,
grantaward.category_niicode,
grantaward.directcost,
grantaward.allocation_niicode
FROM grantaward
WHERE grantaward.category_niicode = 72
OR grantaward.category_niicode = 69
'''
df_main = pd.read_sql_query(sql_main, engine)
df_main.head()
#df_main.describe(include = 'all')

,awardnumber,startfiscalyear,endfiscalyear,category_niicode,directcost,allocation_niicode
0,14310002,2002,2005,69,6800000.0,hojokin
1,14310007,2002,2005,69,13700000.0,hojokin
2,14310011,2002,2005,69,13600000.0,hojokin
3,14310014,2002,2005,69,12900000.0,hojokin
4,14310015,2002,2005,69,6100000.0,hojokin


In [27]:
df_main.awardnumber.duplicated().any() #False
df_main = df_main.set_index("awardnumber")
df_main["len_project"] = df_main["endfiscalyear"] - df_main["startfiscalyear"] +1
product_list = [product_journal_article,product_jointinternational,product_presentation, product_book,product_patent,product_symposium, product_publication ]
df_main_product = df_main.join(product_list).fillna(0)


In [30]:
# Write to csv
pathname = "/Users/norika_machome/GitHub/Capstone_KAKEN"
df_main_product.to_csv(pathname + "/Data/Parsed_csv/" + "df_main_product" + ".csv", index = True)
print("Written to:", pathname + "/Data/Parsed_csv/" + "df_main_product" + ".csv")

Written to: /Users/norika_machome/GitHub/Capstone_KAKEN/Data/Parsed_csv/df_main_product.csv


In [29]:
df_main_product

,startfiscalyear,endfiscalyear,category_niicode,directcost,allocation_niicode,len_project,journal_freq_count,jointInt_freq_count,presentation_freq_count,book_freq_count,patent_freq_count,symposium_freq_count,publication_freq_count
awardnumber,,,,,,,,,,,,,
14310002,2002.0,2005.0,69.0,6800000.0,hojokin,4.0,27.0,0.0,0.0,9.0,0.0,0.0,12.0
14310007,2002.0,2005.0,69.0,13700000.0,hojokin,4.0,20.0,0.0,0.0,15.0,0.0,0.0,3.0
14310011,2002.0,2005.0,69.0,13600000.0,hojokin,4.0,21.0,0.0,0.0,4.0,0.0,0.0,25.0
14310014,2002.0,2005.0,69.0,12900000.0,hojokin,4.0,38.0,0.0,0.0,15.0,0.0,0.0,10.0
14310015,2002.0,2005.0,69.0,6100000.0,hojokin,4.0,23.0,0.0,0.0,2.0,0.0,0.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26520309,2014.0,2016.0,72.0,3700000.0,kikin,3.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0
26520310,2014.0,2016.0,72.0,3800000.0,kikin,3.0,5.0,2.0,24.0,1.0,0.0,0.0,0.0
26520311,2014.0,2016.0,72.0,3700000.0,kikin,3.0,10.0,0.0,7.0,1.0,0.0,0.0,0.0
